# ELEMENTS:

In [2]:
# Created by K.Nakamura on 2015-11-15.
# e-mail: nakamura-keita-kn@ynu.jp


abstract  AbstractElement{DIM}
typealias AbstractElement1D AbstractElement{1}               # Used in Line
typealias AbstractElement2D AbstractElement{2}               # Used in Surface
typealias AbstractElement3D AbstractElement{3}               # Used in Volume

#----------------------------------------------------------------------------------------------------------
# The purpose of these interation is to printout the members of the element
#----------------------------------------------------------------------------------------------------------

# just return element itself for iteration, but a bit confusing due to the definition of length(element)?
Base.start(element::AbstractElement) = 1                     ##########  WHERE IS IT USED?
Base.done(element::AbstractElement, state) = state > 1
Base.next(element::AbstractElement, state) = element, 2

#-----------------------------------------------------------------------------------------------------------
# In FEM, we work with ELEMENTS. Each element has the following properties:
#        
#       For example: Elt1 = Quad4([1,2,3,4])
#       
#        1) connectivity: return The number of nodes in that elements: [1,2,3,4]                        
#              Based on the input vector of connectivity
#
#        2) gauss_points:

#        3) shape functions: includes N:Shape function,∇N: derivatives of shape function:                  
#             Based on the type of "elt_type": Quad4, the constructor of ShapeFunction type (file Element2D.jl,
#             the  shape function and derivative of shape function are returned as the attributes of the
#             ShapeFunction type
#
#        4) field:
#-----------------------------------------------------------------------------------------------------------
immutable Element{DIM, N} <: AbstractElement{DIM}
    connectivity::Vector{Int}
    gauss_points::Vector{GaussPoint}
    shape::ShapeFunction
    fields::FieldSet
    
    function Element(connectivity::Vector)
        @assert length(connectivity) == N
        elt_type = Element{DIM,N}
        return new(
            connectivity,
            create_gauss_points(elt_type),
            ShapeFunction(elt_type),
            FieldSet(),
        )
    end
    function Element(connectivity::Vector, gauss_points::Vector{GaussPoint})
        @assert length(connectivity) == N
        elt_type = Element{DIM,N}
        return new(
            connectivity,
            gauss_points,
            ShapeFunction(elt_type),
            FieldSet(),
        )
    end
end

#-----------------------------------------------------------------------------------------------------------
# Purpose: Return the number of nodes in the element
function Base.length{DIM,N}(element::Element{DIM,N})
    return N
end

#
function field_length(element::Element)
    return length(element["geometry"]) 
end

#------------------------------------------------------------------------------------------------------------
# Purpose: return the dimension of the element
function get_dimension{DIM,N}(element::Element{DIM,N})
    return DIM
end

#-------------------------------------------------------------------------------------------------------------
# Purpose: return the nodes' numbers
function get_connectivity(element::Element)
    return element.connectivity
end


function get_shape_function(element::Element)
    return element.shape
end

#--------------------------------------------------------------------------------------------------------------
# Purpose: return the coordinates of Gauss_points in the element
function get_gauss_points(element::Element)
    return element.gauss_points
end

function get_fieldset(element::Element)
    return element.fields
end

function Base.getindex(element::Element, field_name)
    return element.fields[field_name]
end

function Base.setindex!(element::Element, field_data, field_name)
    setindex!(element.fields, field_data, field_name)
end

function Base.haskey(element::Element, key)
    return haskey(element.fields, key)
end

function Base.keys(element::Element)
    return keys(element.fields)
end

function get_dofs(element::Element, dim::Int)
    conn = get_connectivity(element)
    return get_dofs(conn, dim)
end

function get_dofs(element::Element, dim::Int, target)
    conn = get_connectivity(element)
    return get_dofs(conn, dim, target)
end

function get_dofs(elements::Union{Vector,Base.Generator}, dim::Int, args...)
    dofs = Int64[]
    for elt in elements
        append!(dofs, get_dofs(elt, dim, args...))
    end
    return unique(dofs)
end

function next_timestep!(element::Element, Δt::Real=1.0)
    next_timestep!(get_fieldset(element), Δt)
    for gp in get_gauss_points(element)
        next_timestep!(gp, Δt)
    end
end

function pop_fields!(element::Element)
    pop_fields!(get_fieldset(element), Δt)
    for gp in get_gauss_points(element)
        pop_fields!(gp)
    end
end

function shift_fields!(element::Element) # This function looks dangerous, should be deleted?
    shift_fields!(get_fieldset(element), Δt)
    for gp in get_gauss_points(elt)
        shift_fields!(gp)
    end
end

function (element::Element)(point)
    shape = get_shape_function(element)
    return shape(point)
end

function (element::Element)(point, dim)
    shape = get_shape_function(element)
    return shape(point, dim)
end
#--------------------------------------- WHAT IS THE PURPOSE OF THIS PART ? -------------------------------------------
for NAME in ["GRAD", "JACOBIAN", "detJ"]
    Val_NAME = Val{Symbol(NAME)}
    Val_name = Val{Symbol(lowercase(NAME))}
    @eval begin
        # use reference configuration
        function (element::Element)(::Type{$Val_NAME}, point)
            shape = get_shape_function(element)
            geometry = element["geometry"][1]
            return shape($Val_name, point, geometry)
        end
        # use current configuration
        function (element::Element)(val::Type{$Val_name}, point; time=:end)
            shape = get_shape_function(element)
            geometry = element["geometry"][time]
            return shape($Val_name, point, geometry)
        end
    end
end
#---------------------------------------------------------------------------------------------------------
#THIS IS SIMILAR TO THE FOLLOWING CODE, jUST TO SHORTEN THE CODE
#  function (element::Element)(::Type{:GRAD}, point)
#            shape = get_shape_function(element)
#            geometry = element["geometry"][1]
#            return shape(:grad, point, geometry)
#  end
#
#  function (element::Element)(::Type{:JACOBIAN}, point)
#            shape = get_shape_function(element)
#            geometry = element["geometry"][1]
#            return shape(:jacobian, point, geometry)
#  end
#
#  function (element::Element)(::Type{:detJ}, point)
#            shape = get_shape_function(element)
#            geometry = element["geometry"][1]
#            return shape(:detj, point, geometry)
#  end
#
#  function (element::Element)(::Type{:grad}, point)
#            shape = get_shape_function(element)
#            geometry = element["geometry"][time]
#            return shape(:grad, point, geometry)
#  end
#
#  function (element::Element)(::Type{:jacobian}, point)
#            shape = get_shape_function(element)
#            geometry = element["geometry"][time]
#            return shape(:grad, point, geometry)
#  end
#
#  function (element::Element)(::Type{:detj}, point)
#            shape = get_shape_function(element)
#            geometry = element["geometry"][time]
#            return shape(:detj, point, geometry)
#----------------------------------------------------------------------------------------------------------------------
function (element::Element)(fieldname::String, point; time=:end)
    val = element[fieldname][time]
    N = element(point)
    if size(N,2) == size(val,1) # this is not good?
        return (N * val)[1]
    else
        return val
    end
end

function (element::Element)(fieldname::String, ::Type{Val{:GRAD}}, point; time=:end)
    B = element(Val{:GRAD}, point)
    return hcat(B * element[fieldname][time]...)'
end

# -----------------------------------------SEE MACRO-GENERATED FUNCTION -----------------------------------------------
@generated function (element::Element)(fieldname::String, ::Type{Val{:grad}}, point; time::Union{Real,Symbol,Pair}=:end)
    if time <: Pair
        return quote
            B = element(Val{:grad}, point; time=time.second)
            hcat(B * element[fieldname][time]...)'
        end
    else
        return quote
            B = element(Val{:grad}, point; time=ifelse(time==:dend, :end, time))
            hcat(B * element[fieldname][time]...)'
        end
    end
end


include("element1d.jl")
include("element2d.jl")
include("element3d.jl")


LoadError: LoadError: UndefVarError: GaussPoint not defined
while loading In[2], in expression starting on line 16